# TP1: Methodes directes pour la résolution des systèmes linéaires

On souhaite résoudre l'équation de la forme: 

## $$ Ax = b $$

Avec, A une matrice carrée de dimension (n,n), x de dimension (n,1) et b de dimension (n,1). la matrice A et le vecteur b sont connues et on cherche à trouver x. d'ordinaire on aurait trouvé l'inverse de la matrics A afin d'en déduire x avec l'équation $$ x = A^{-1}b $$  

Trouver l'inverse d'une matrice devient vite compliqué à calculer lorsque la taille de la matrice augmente, ceci n'étant pas scalable, il faut utiliser une autre méthode pour résoudre cette équation plus rapidement.

In [40]:
import numpy as np
import scipy as sc

from scipy import linalg as lg

In [41]:
def gen_a_b(n=5):
    # On génère une matrice inversible (det(A) = 0) et qui n'aura pas de permutation dans la decomp LU
    while True:  
        A = np.random.randint(5,size=(n, n))
        p,L, U = lg.lu(A)

        if np.linalg.det(A) != 0 and np.trace(p) == n:
            break

    b = np.random.randint(5,size=(n, 1))
    
    return A,b

A,b = gen_a_b(5)
A

array([[4, 3, 2, 3, 1],
       [4, 0, 2, 0, 0],
       [2, 4, 0, 0, 0],
       [4, 1, 2, 3, 2],
       [1, 1, 0, 0, 3]])

In [42]:
b

array([[1],
       [0],
       [3],
       [3],
       [3]])

on regarde si la matrice est inversible, elle ne l'est pas si il y a des 0
https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.inv.html

## Decomposition LU

In [43]:
from scipy import linalg as lg

p,L, U = lg.lu(A, )



P est une matrice de permutation

https://www.wikiwand.com/en/Permutation_matrix

In [44]:
p

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [45]:
L

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 1.        ,  1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.5       , -0.83333333,  1.        ,  0.        ,  0.        ],
       [ 1.        ,  0.66666667, -0.        ,  1.        ,  0.        ],
       [ 0.25      , -0.08333333,  0.5       ,  0.5       ,  1.        ]])

In [46]:
U

array([[ 4.        ,  3.        ,  2.        ,  3.        ,  1.        ],
       [ 0.        , -3.        ,  0.        , -3.        , -1.        ],
       [ 0.        ,  0.        , -1.        , -4.        , -1.33333333],
       [ 0.        ,  0.        ,  0.        ,  2.        ,  1.66666667],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  2.5       ]])

### Decomp LU sans librairie

In [47]:
def decomp_lu(A):
    """ Calcule la décomposition LU d'une matrice carré inversible A
    """
    n = A.shape[0]
    
    L = np.zeros((n,n))
    U = np.zeros((n,n))
    
    # 1ere ligne de U = 1ere ligne de A
    U[0] = A[0]
        
    # 1ere colonne de L = 1ere colonne de A
    L[:,0] = A[:,0] / U[0,0]
    
    for i in range(1, n):
        L[i,i] = 1
        U[i,i] = A[i,i] - np.sum(np.multiply(L[i,:i], U[:i,i]))
        
        for j in range(i + 1, n):
            U[i,j] = A[i,j] - np.sum(np.multiply(L[i,:i], U[:i,j]))
            L[j,i] = (A[j,i] - np.sum(np.multiply(L[j,:i], U[:i,i]))) / U[i,i]
        
    return L,U
    
Lm,Um = decomp_lu(A)

In [48]:
Lm 


array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 1.        ,  1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.5       , -0.83333333,  1.        ,  0.        ,  0.        ],
       [ 1.        ,  0.66666667, -0.        ,  1.        ,  0.        ],
       [ 0.25      , -0.08333333,  0.5       ,  0.5       ,  1.        ]])

In [49]:
L

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 1.        ,  1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.5       , -0.83333333,  1.        ,  0.        ,  0.        ],
       [ 1.        ,  0.66666667, -0.        ,  1.        ,  0.        ],
       [ 0.25      , -0.08333333,  0.5       ,  0.5       ,  1.        ]])

In [50]:
Um

array([[ 4.        ,  3.        ,  2.        ,  3.        ,  1.        ],
       [ 0.        , -3.        ,  0.        , -3.        , -1.        ],
       [ 0.        ,  0.        , -1.        , -4.        , -1.33333333],
       [ 0.        ,  0.        ,  0.        ,  2.        ,  1.66666667],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  2.5       ]])

In [51]:
U

array([[ 4.        ,  3.        ,  2.        ,  3.        ,  1.        ],
       [ 0.        , -3.        ,  0.        , -3.        , -1.        ],
       [ 0.        ,  0.        , -1.        , -4.        , -1.33333333],
       [ 0.        ,  0.        ,  0.        ,  2.        ,  1.66666667],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  2.5       ]])

## Triangulaire inférieure

In [52]:
def triang_inf(T,b):
    """calcule x selon l'équation Tx=b avec T une matrice triangulaire inférieure et b un vecteur connu
    Return:
        x: vecteur solution de l'équation
    """
    n = T.shape[0]
    x = np.zeros((n, 1))

    for k in range(n):
        x[k] = (b[k] - np.sum(np.multiply(T[k], np.transpose(x))))/ T[k,k]
    
    return x
    
y = triang_inf(L,b)

In [53]:
np.dot(L,y)

array([[1.],
       [0.],
       [3.],
       [3.],
       [3.]])

In [54]:
b

array([[1],
       [0],
       [3],
       [3],
       [3]])

## Triangulaire supérieure

In [55]:
def triang_sup(T,b):
    """calcule x selon l'équation Tx=b avec T une matrice triangulaire supérieure et b un vecteur connu
    Return:
        x: vecteur solution de l'équation
    """
    n = T.shape[0]
    x = np.zeros((n, 1))

    for k in range(n - 1 , -1, -1):
        x[k] = (b[k] - np.sum(np.multiply(T[k], np.transpose(x)))) / T[k,k]
    
    
    return x
    
x = triang_sup(U,y)
    

In [56]:
np.dot(np.transpose(p),np.dot(A,x))

array([[1.],
       [0.],
       [3.],
       [3.],
       [3.]])

In [57]:
np.dot(np.transpose(np.dot(A,x)), p)

array([[1., 0., 3., 3., 3.]])

In [58]:
b

array([[1],
       [0],
       [3],
       [3],
       [3]])

## Pivot de Gauss

In [59]:
def gauss(A, b):
    n = A.shape[0]
    M = A
    x = np.zeros((n, 1))
    
    for k in range(n - 1):
        for i in range(k + 1, n):
            M[i, k] = A[i, k] / A[k, k]
            A[i, k:] = A[i, k:] - M[i, k] * A[k, k:]
            b[i] = b[i] - M[i, k] * b[k]
    
    for i in range(n - 1 , -1, -1):
        x[i] = (b[i] - np.sum(np.multiply(A[i], np.transpose(x)))) / A[i, i]
        
    return x

xg = gauss(A, b)
xg

/home/antoine/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in long_scalars
  


ValueError: cannot convert float NaN to integer

In [ ]:
np.dot(A,xg)

# Problèmes avec l'inverse d'une matrice

se baser sur trouver l'inverse de la matrice n'est pas pratique car la complexité croit exponentiellement lorsqu'on augmente la taille de la matrice. Il n'est pas du tout pratique d'utiliser cette méthode mais plutot d'autres techniques de decomposition nettement plus rapides qui auront un temp plus linéaire à la tâche à accomplir

In [ ]:
sizes = [10,25,50,100,200] # avec 10000 ca prend trop longtemp, 42s cossomant 4gb de RAM et 100% du processeur
#on teste les tailles différentes
for size in sizes:
    A1 = np.random.randint(3,size=(size, size))
    print("taille de",size ,"inversion")
    %timeit -n 1 np.linalg.inv(A1)
    

lorsqu'on passe d'une matrice de taille 1000 à 10000, le temps d'inversion passe de 240 ms à 42s! tout cela en cossomant beaucoup de mémoire (presque 4Gb de RAM) et 100% du processeur

# Test des autre méthodes
voici un test des autres méthodes de résolution afin de comparer avec l'inversion d'une matrice vu ci dessus.

1) decomp lu

In [ ]:
sizes = [10,25,50,100,200]
#on teste les tailles différentes
for size in sizes:
    print("taille de",size)
    A1 = np.random.randint(3,size=(size, size))
    print("decomp")
    %timeit -n 1 lg.lu(A1)
    

2) Pivot de gauss

In [ ]:
sizes = [10,25,50,100,200]
#on teste les tailles différentes
for size in sizes:
    print("taille de",size)
    A1 = np.random.randint(3,size=(size, size))
    b1 = np.random.randint(3,size=(size))
    print("decomp")
   # %timeit -n 1 gauss(A1,b1)

## 5 Aller plus loin

### Décomposition de Cholesky 

La méthode de cholesky permet de factoriser une matric A __Symétrique définie positive__ sous la forme $ A=CC^T$ où C est une matrice triangulaire inférieure inversible



In [ ]:
A = np.array([[15.0,10.0,18.0,12.0],
              [10.0,15.0,7.0 ,13.0],
              [18.0,7.0 ,27.0,7.0 ],
              [12.0,13.0,7.0 ,22.0]])



#### Calculer la matrice C, est ce que A est définie positive

$l_{kk} = \sqrt{ a_{kk} - \sum^{k-1}_{j=1} l^2_{kj}}$ 

$ l_{ik} = \frac{1}{l_{kk}} \left( a_{ik} - \sum^{k-1}_{j=1} l_{ij} l_{kj} \right) $

In [ ]:
from math import sqrt

def cholesky(A):
    """ calcule la matrice C correspondant à la 
        décomposition de cholesky d'une matrice A
        A doit etre une matrice symétrique positive définie
    """
    n = A.shape[0]
    
    C = np.zeros((n,n))
    
    C[0,0] = sqrt(A[0,0])
    
    C[1:,0] = A[1:,0] / C[0,0]
    
    
    
    for j in range(1,n):
        for i in range(j+1):
            
            tmp_sum = np.sum(np.multiply(C[i,:i],C[j,:i]))
            
            
            #diagonal coefs
            if i == j:
                C[j,i] = sqrt( (A[i,j] - tmp_sum ) ) # j > 1
            else:
            #other
                C[j,i] = (A[i,j] - tmp_sum ) / (C[i,i])
            
            
    return C
            
C = cholesky(A)

C

In [ ]:
Cl = np.linalg.cholesky(A)
Cl

In [ ]:
np.dot(C,np.transpose(C)) #this gives us A

### Est ce que A est définie positive
Par définition, A est dite définie positive si toutes ses valeurs propres sont strictement positives, puisque il n'est pas forcément facile de calculer les valeurs propres on peut utiliser la condition de Sylvester qui consiste à regarder les mienurs principaux de la matrice et de s'assurer que ces derniers sont tous strictement positifs.

Les mineurs principaux correspondent aux déterminants de la matrice tronqué.


par def la decomp cholesky nous donne une matrice C qui est définie par la relation $A= CC^T$

On peut tenter de trouver la solution du système suivant : $Ax = b$

In [ ]:
b = np.array([53.0,72.0,26.0,97.0])

y1 = triang_inf(C,b)

In [ ]:
y1

In [ ]:
np.dot(C,y1) #should give us b

In [ ]:
x1 = triang_sup(np.transpose(C),y1)

x1

In [ ]:
np.linalg.solve(A,b)

In [ ]:
np.dot(A,x1)

## Decomposition de Ritchmayer

In [ ]:
from scipy.sparse import diags

# Générons une matrice tridiagonale
a = [1, 2, 1, 2]
b = [2, 3, 1, 1, 1]
c = [3, 1, 2, 1]

A = diags([a, b, c], [-1, 0, 1], shape=(5, 5), dtype=int).toarray() #genere la matrice trigiagonale
print(A)

# Générons un vecteur u
u = np.array([-1, 0, 0, 3, 1])
print(u)

In [ ]:
def rich(a, b, c, u):
    """
        Exécute l'algorithme de Richtmayer et retourne le vecteur x, solution de l'équation Ax = u, avec A
        une matrice tridiagonale.
    """
    n = len(b)
    e = [0, 0, 0, 0, 0]
    f = [0, 0, 0, 0, 0]
    x = [0, 0, 0, 0, 0]
    
    e[0] = -c[0] / b[0]
    f[0] = u[0] / b[0]
    
    for i in range(1, n - 1):
        e[i] = -c[i] / (a[i - 1] * e[i - 1] + b[i])
        
    for i in range(1, n):
        f[i] = (u[i] - a[i - 1] * f[i - 1]) / (a[i - 1] * e[i - 1] + b[i])
        
    x[n - 1] = f[n - 1]
    for i in range(n - 2, -1, -1):
        x[i] = e[i] * x[i + 1] + f[i]
        
    return x

# On exécute l'algo et on convertit les résultats en int
xr = [int(i) for i in rich(a, b, c, u)]

In [ ]:
np.dot(A,xr)

In [ ]:
x_sol = np.linalg.solve(A, u)

In [ ]:
np.dot(A,x_sol)